# ✔ CUDA setup

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [7]:
!nvidia-smi

Tue Feb 22 17:37:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-d35i0k5t
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-d35i0k5t
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=6e2074ae42aaa2c03af2741fbe35603c3dbe61fc287a9ff0b484eba62888f8db
  Stored in directory: /tmp/pip-ephem-wheel-cache-8a7dzwgc/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

directory /content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel/src already exists
Out bin /content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel/result.out


# ✔ VS Code on Colab

In [ ]:
# 1. Install the colab-code package...
!pip install colabcode

In [ ]:
# 2. Import and launch...
from colabcode import ColabCode
ColabCode()

Code Server can be accessed on: NgrokTunnel: "https://1636-35-185-135-22.ngrok.io" -> "http://localhost:10000"
[2021-10-16T17:00:56.429Z] info  code-server 3.10.2 387b12ef4ca404ffd39d84834e1f0776e9e3c005
[2021-10-16T17:00:56.431Z] info  Using user-data-dir ~/.local/share/code-server
[2021-10-16T17:00:56.450Z] info  Using config file ~/.config/code-server/config.yaml
[2021-10-16T17:00:56.450Z] info  HTTP server listening on http://127.0.0.1:10000 
[2021-10-16T17:00:56.450Z] info    - Authentication is disabled 
[2021-10-16T17:00:56.450Z] info    - Not serving HTTPS 
 INFO Installing extension: ms-vscode.cpptools
 INFO Downloaded extension: ms-vscode.cpptools /root/.local/share/code-server/CachedExtensionVSIXs/ms-vscode.cpptools-1.5.1
 INFO Extracted extension to /root/.local/share/code-server/extensions/.801240b6-e04b-4087-9084-3b6369662ed5: ms-vscode.cpptools
 INFO Renamed to /root/.local/share/code-server/extensions/ms-vscode.cpptools-1.5.1
 INFO Installation completed. ms-vscode.cppt

#DeviceQuery

In [ ]:
%%writefile helper.h
// Beginning of GPU Architecture definitions
inline int _ConvertSMVer2Cores(int major, int minor) {
	// Defines for GPU Architecture types (using the SM version to determine
	// the # of cores per SM
	typedef struct {
		int SM;  // 0xMm (hexidecimal notation), M = SM Major version,
		// and m = SM minor version
		int Cores;
	} sSMtoCores;

	sSMtoCores nGpuArchCoresPerSM[] = {
			{0x20, 32},
			{0x30, 192},
			{0x32, 192},
			{0x35, 192},
			{0x37, 192},
			{0x50, 128},
			{0x52, 128},
			{0x53, 128},
			{0x60,  64},
			{0x61, 128},
			{0x62, 128},
			{0x70,  64},
			{0x72,  64},
			{0x75,  64},
			{-1, -1}};

	int index = 0;

	while (nGpuArchCoresPerSM[index].SM != -1) {
		if (nGpuArchCoresPerSM[index].SM == ((major << 4) + minor)) {
			return nGpuArchCoresPerSM[index].Cores;
		}

		index++;
	}

	// If we don't find the values, we default use the previous one
	// to run properly
	printf(
			"MapSMtoCores for SM %d.%d is undefined."
			"  Default to use %d Cores/SM\n",
			major, minor, nGpuArchCoresPerSM[index - 1].Cores);
	return nGpuArchCoresPerSM[index - 1].Cores;
}


Writing helper.h


In [ ]:
%%writefile deviceQuery.cu
#include <stdlib.h>
#include <stdio.h>
#include "helper.h"
#include "Utils.h"

int main(void) {

	printf("\nCUDA Device Query (Runtime API) version (CUDART static linking)\n\n");
	int deviceCount = 0;
	CHECK(cudaGetDeviceCount(&deviceCount));

	// This function call returns 0 if there are no CUDA capable devices.
	if (deviceCount == 0)
		printf("There are no available device(s) that support CUDA\n");
	else
		printf("Detected %d CUDA Capable device(s)\n", deviceCount);

	int dev, driverVersion = 0, runtimeVersion = 0;

	for (dev = 0; dev < deviceCount; ++dev) {
		cudaSetDevice(dev);
		cudaDeviceProp deviceProp;
		cudaGetDeviceProperties(&deviceProp, dev);

		printf("\nDevice %d: \"%s\"\n", dev, deviceProp.name);

		cudaDriverGetVersion(&driverVersion);
		cudaRuntimeGetVersion(&runtimeVersion);

		printf("  CUDA Driver Version / Runtime Version          %d.%d / %d.%d\n",
				driverVersion / 1000, (driverVersion % 100) / 10,
				runtimeVersion / 1000, (runtimeVersion % 100) / 10);

		printf("  CUDA Capability Major/Minor version number:    %d.%d\n",
				deviceProp.major, deviceProp.minor);

		printf("  Total amount of global memory:                 %.0f MBytes (%llu bytes)\n",
				(float) deviceProp.totalGlobalMem / 1048576.0f,
				(unsigned long long) deviceProp.totalGlobalMem);

	    printf("  (%2d) Multiprocessors, (%3d) CUDA Cores/MP:     %d CUDA Cores\n",
	           deviceProp.multiProcessorCount,
	           _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor),
	           _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor) *
	               deviceProp.multiProcessorCount);

		printf("  GPU Max Clock rate:                            %.0f MHz (%0.2f GHz)\n",
				deviceProp.clockRate * 1e-3f, deviceProp.clockRate * 1e-6f);

		printf("  Memory Clock rate:                             %.0f Mhz\n", deviceProp.memoryClockRate * 1e-3f);
		printf("  Memory Bus Width:                              %d-bit\n", deviceProp.memoryBusWidth);
		if (deviceProp.l2CacheSize)
			printf("  L2 Cache Size:                                 %d bytes\n", deviceProp.l2CacheSize);

		printf("  Maximum Texture Dimension Size (x,y,z)         1D=(%d), 2D=(%d, %d), 3D=(%d, %d, %d)\n",
				deviceProp.maxTexture1D, deviceProp.maxTexture2D[0],
				deviceProp.maxTexture2D[1], deviceProp.maxTexture3D[0],
				deviceProp.maxTexture3D[1], deviceProp.maxTexture3D[2]);

		printf("  Maximum Layered 1D Texture Size, (num) layers  1D=(%d), %d layers\n",
				deviceProp.maxTexture1DLayered[0],
				deviceProp.maxTexture1DLayered[1]);

		printf("  Maximum Layered 2D Texture Size, (num) layers  2D=(%d, %d), %d layers\n",
				deviceProp.maxTexture2DLayered[0],
				deviceProp.maxTexture2DLayered[1],
				deviceProp.maxTexture2DLayered[2]);

		printf("  Total amount of constant memory:               %lu bytes\n",
				deviceProp.totalConstMem);
		printf("  Total amount of shared memory per block:       %lu bytes\n",
				deviceProp.sharedMemPerBlock);
		printf("  Total number of registers available per block: %d\n",
				deviceProp.regsPerBlock);
		printf("  Warp size:                                     %d\n",
				deviceProp.warpSize);
		printf("  Maximum number of threads per multiprocessor:  %d\n",
				deviceProp.maxThreadsPerMultiProcessor);
		printf("  Maximum number of threads per block:           %d\n",
				deviceProp.maxThreadsPerBlock);
		printf("  Max dimension size of a thread block (x,y,z): (%d, %d, %d)\n",
				deviceProp.maxThreadsDim[0], deviceProp.maxThreadsDim[1],
				deviceProp.maxThreadsDim[2]);
		printf("  Max dimension size of a grid size    (x,y,z): (%d, %d, %d)\n",
				deviceProp.maxGridSize[0], deviceProp.maxGridSize[1],
				deviceProp.maxGridSize[2]);
		printf("  Maximum memory pitch:                          %lu bytes\n",
				deviceProp.memPitch);
		printf("  Texture alignment:                             %lu bytes\n",
				deviceProp.textureAlignment);
		printf("  Concurrent copy and kernel execution:          %s with %d copy engine(s)\n",
				(deviceProp.deviceOverlap ? "Yes" : "No"),
				deviceProp.asyncEngineCount);
		printf("  Run time limit on kernels:                     %s\n",
				deviceProp.kernelExecTimeoutEnabled ? "Yes" : "No");
		printf("  Integrated GPU sharing Host Memory:            %s\n",
				deviceProp.integrated ? "Yes" : "No");
		printf("  Support host page-locked memory mapping:       %s\n",
				deviceProp.canMapHostMemory ? "Yes" : "No");
		printf("  Alignment requirement for Surfaces:            %s\n",
				deviceProp.surfaceAlignment ? "Yes" : "No");
		printf("  Device has ECC support:                        %s\n",
				deviceProp.ECCEnabled ? "Enabled" : "Disabled");

		printf("  Device supports Unified Addressing (UVA):      %s\n",
				deviceProp.unifiedAddressing ? "Yes" : "No");
		printf("  Device PCI Domain ID / Bus ID / location ID:   %d / %d / %d\n",
				deviceProp.pciDomainID, deviceProp.pciBusID,
				deviceProp.pciDeviceID);
	}
	return 0;
}


Overwriting deviceQuery.cu


In [ ]:
# Compilation and execution

!nvcc deviceQuery.cu -o deviceQuery
!./deviceQuery


CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla K80"
  CUDA Driver Version / Runtime Version          11.2 / 11.1
  CUDA Capability Major/Minor version number:    3.7
  Total amount of global memory:                 11441 MBytes (11996954624 bytes)
  (13) Multiprocessors, (192) CUDA Cores/MP:     2496 CUDA Cores
  GPU Max Clock rate:                            824 MHz (0.82 GHz)
  Memory Clock rate:                             2505 Mhz
  Memory Bus Width:                              384-bit
  L2 Cache Size:                                 1572864 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(65536), 2D=(65536, 65536), 3D=(4096, 4096, 4096)
  Maximum Layered 1D Texture Size, (num) layers  1D=(16384), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(16384, 16384), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total amount of shared memory per block:       49152

# ✔ Profiling

In [ ]:
# Query
!nvprof --query-events
!nvprof --query-metrics

Available Metrics:
                            Name   Description
Device 0 (Tesla K80):
        l1_cache_global_hit_rate:  Hit rate in L1 cache for global loads

         l1_cache_local_hit_rate:  Hit rate in L1 cache for local loads and stores

                   sm_efficiency:  The percentage of time at least one warp is active on a multiprocessor averaged over all multiprocessors on the GPU

                             ipc:  Instructions executed per cycle

              achieved_occupancy:  Ratio of the average active warps per active cycle to the maximum number of warps supported on a multiprocessor

        gld_requested_throughput:  Requested global memory load throughput

        gst_requested_throughput:  Requested global memory store throughput

          sm_efficiency_instance:  The percentage of time at least one warp is active on a specific multiprocessor

                    ipc_instance:  Instructions executed per cycle for a single multiprocessor

            inst_repl

In [6]:
# Compilation (e.g. -arch=sm_37 if compute capability is 3.7)
!nvcc -arch=sm_37 FlockingBehaviourPar.cu FlockingBehaviourSeq.cpp Utils.cpp -lcurand -o FlockingBehaviourPar

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [1]:
# Occupancy
!nvprof --metrics achieved_occupancy,branch_efficiency ./FlockingBehaviourPar

Available Metrics:
                            Name   Description
Device 0 (Tesla K80):
        l1_cache_global_hit_rate:  Hit rate in L1 cache for global loads

         l1_cache_local_hit_rate:  Hit rate in L1 cache for local loads and stores

                   sm_efficiency:  The percentage of time at least one warp is active on a multiprocessor averaged over all multiprocessors on the GPU

                             ipc:  Instructions executed per cycle

              achieved_occupancy:  Ratio of the average active warps per active cycle to the maximum number of warps supported on a multiprocessor

        gld_requested_throughput:  Requested global memory load throughput

        gst_requested_throughput:  Requested global memory store throughput

          sm_efficiency_instance:  The percentage of time at least one warp is active on a specific multiprocessor

                    ipc_instance:  Instructions executed per cycle for a single multiprocessor

            inst_repl

In [11]:
# Divergence
#!nvprof --metrics branch_efficiency ./FlockingBehaviourPar
!nvprof --events branch,divergent_branch ./FlockingBehaviourPar

======== Warning: Event "branch" cannot be found on device 0.
======== Warning: Event "divergent_branch" cannot be found on device 0.
==668== NVPROF is profiling process 668, command: ./FlockingBehaviourPar
device 0: Tesla K80

Flock dimension: 40960
Neighborhood dimension: 50.00
Velocity: 20.00
Update time: 1.50
Iterations: 1
Separation weight: 1.00
Cohesion weight: 1.00
Align weight: 1.00
Minimum random number: -50000
Maximum random number: 50000
Decimal digits of random numbers: 3.00
==668== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.

Needed threads number: 2048
Threads used: 2048
Block size: 128
Grid size: 16
Generations per thread: 20
Generations of last thread: 20


GPU Flock generation...
==668== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the r

In [8]:
# Transfers
#!nvprof --metrics gld_efficiency,gst_efficiency ./FlockingBehaviourPar
!nvprof --metrics shared_efficiency,shared_load_transactions_per_request,shared_store_transactions_per_request --events shared_load_replay,shared_store_replay ./FlockingBehaviourPar

==597== NVPROF is profiling process 597, command: ./FlockingBehaviourPar
device 0: Tesla K80

Flock dimension: 40960
Neighborhood dimension: 50.00
Velocity: 20.00
Update time: 1.50
Iterations: 1
Separation weight: 1.00
Cohesion weight: 1.00
Align weight: 1.00
Minimum random number: -50000
Maximum random number: 50000
Decimal digits of random numbers: 3.00
==597== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.

Needed threads number: 2048
Threads used: 2048
Block size: 128
Grid size: 16
Generations per thread: 20
Generations of last thread: 20


GPU Flock generation...
==597== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==597== Some kernel(s) will be replayed on device 0 in order to collect all events/metrics.
==597== Replaying

In [8]:
# All
!nvprof --metrics achieved_occupancy,branch_efficiency,gld_efficiency,gst_efficiency,shared_efficiency,shared_load_transactions_per_request,shared_store_transactions_per_request --events shared_load_replay,shared_store_replay ./FlockingBehaviourPar
#!nvprof --metrics all --events all ./FlockingBehaviourPar

==404== NVPROF is profiling process 404, command: ./FlockingBehaviourPar
device 0: Tesla K80

Flock dimension: 40960
Neighborhood dimension: 50.00
Velocity: 20.00
Update time: 1.50
Iterations: 1
Separation weight: 1.00
Cohesion weight: 1.00
Align weight: 1.00
Minimum random number: -50000
Maximum random number: 50000
Decimal digits of random numbers: 3.00
==404== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.

Needed threads number: 2048
Threads used: 2048
Block size: 128
Grid size: 16
Generations per thread: 20
Generations of last thread: 20


GPU Flock generation...
==404== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==404== Some kernel(s) will be replayed on device 0 in order to collect all events/metrics.
==404== Replaying

In [10]:
# Spilling
!nvcc --ptxas-options=-v -arch=sm_37 FlockingBehaviourPar.cu

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc error   : 'ptxas' died due to signal 2 


# Flocking Behaviour Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd "content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourSequential"
%ls

/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourSequential
Boid.cpp  Flock.cpp  FlockingBehaviourSeq     FlockingBehaviourSeq.h  Utils.cpp
Boid.h    Flock.h    FlockingBehaviourSeq.cu  src/                    Utils.h


In [ ]:
%cd ..
%cd "FlockingBehaviourParallel"
%ls

/
[Errno 2] No such file or directory: 'FlockingBehaviourParallel'
/
bin/      dev/   lib32/  opt/         run/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  proc/        sbin/  tmp/
content/  home/  media/  python-apt/  srv/   tools/
datalab/  lib/   mnt/    root/        sys/   usr/


In [ ]:
%cd ..
%cd "FlockingBehaviourSequential"
%ls

/content/drive/MyDrive/GPU Computing/Progetto
/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourSequential
Boid.cpp  Flock.cpp  FlockingBehaviourSeq     FlockingBehaviourSeq.h  Utils.cpp
Boid.h    Flock.h    FlockingBehaviourSeq.cu  src/                    Utils.h


In [ ]:
%%writefile FlockingBehaviourSeq.h
#include "Boid.h"
#include "Flock.h"
#include "Utils.h"
#include <stdlib.h>
#include <time.h> 
#include <iostream> 
#include <map> 
#include <vector> 

typedef struct BoidData {
    float p1;
    float p2;
    float p3;
    float d1;
    float d2;
    float d3;
} BoidData;

void computeNeighborhoods();

void updateFlock(float);
void getSeparationDirection(int, float*);
void getCohesionDirection(int, float*);
void getAlignDirection(int, float*);
void moveBoid(int, float);

void printNeighborhoods();
void printFlock();

Overwriting FlockingBehaviourSeq.h


In [ ]:
%%writefile FlockingBehaviourSeq.cu
#include "FlockingBehaviourSeq.h"

float velocity = 20; // boid velocity in meters per second
double updateTime = 1.5; // update time of the simulation in seconds
float separationWeigth = 1; // weight of the separation component in the blending
float cohesionWeigth = 1; // weight of the cohesion component in the blending
float alignWeigth = 1; // weight of the align component in the blending
int flockDim = 10000; // 10000000 number of boids in the flock
float neighDim = 200; // 75000000 dimension of the neighborhood in meters
int minRand = -50000; // -50000 minimum value that can be generated for initial position and direction
int maxRand = 50000; // 50000 maximum value that can be generated for initial position and direction
float decimals = 3; // 3 number of decimal digits in the generated values for initial position and direction
int iterations = 1; // number of updates 

float* flockData;
bool* neighborhoods;

int main(void) {

	srand (time(NULL));
	float div = pow(10.0, decimals);
	int numsToGenerate = flockDim * 6;

	// generate boids with random initial position and direction and add them to the flock
	printf("\n\nCPU Flock generation...\n");
	double cpuTimeStart = seconds();

	flockData = (float*) malloc(numsToGenerate * sizeof(float));
	for(int i = 0; i < numsToGenerate; i+=6){
			flockData[i] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+1] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+2] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+3] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+4] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+5] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
		
			vector3Normalize(flockData+i+3);
	}

	double cpuTime = seconds() - cpuTimeStart;
	printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

	for(int i = 0; i < 12; i++){
			printf("first: %.5f\n", flockData[i]);
	}
	for(int i = 0; i < 12; i++){
			printf("last: %.5f\n", flockData[numsToGenerate - i - 1]);
	}

	// calculate neighborhoods (data stored in a boolean matrix)
  neighborhoods = (bool*) malloc(flockDim * flockDim * sizeof(bool));

	printf("\n\nCPU neighborhoods generation...\n");
	cpuTimeStart = seconds();
	computeNeighborhoods();
	cpuTime = seconds() - cpuTimeStart;
	printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

	//printFlock();		
	//std::cout << std::endl;
	//printNeighborhoods();

	// start simulation loop that updates the flock each updateTime
	double loopStart = seconds();
	double tmpTime = updateTime;
	while(iterations > 0){

	 		auto duration = seconds() - loopStart;
	 		if(duration >= tmpTime)
			{
				 	printf("\n\nCPU Flock update...\n");
					cpuTimeStart = seconds();
					updateFlock(updateTime);
					cpuTime = seconds() - cpuTimeStart;
					printf("    CPU elapsed time: %.5f (sec)", cpuTime);
				
					tmpTime += updateTime;
					iterations--;

					//std::cout << std::endl;
				  //printFlock();
				
					//std::cout << std::endl;
					//printNeighborhoods();
			}
	}

	free(flockData);
	free(neighborhoods);
	
	return 0;
}

void computeNeighborhoods(){
    
    for(int i = 0; i < flockDim; i++){
				for(int j = 0; j < flockDim; j++){
						if(j > i){	
								neighborhoods[i*flockDim+j] = vector3Distance(flockData+i*6, flockData+j*6) <= neighDim;
						}
						else if(i == j){
								neighborhoods[i*flockDim+j] = 0;
						}
						else{
								neighborhoods[i*flockDim+j] = neighborhoods[j*flockDim+i];
						}
				}
    }
}

void printFlock(){
    
    for(int i = 0; i < flockDim; i++){
				std::cout << std::setprecision(4) << "Boid " << i << ": " << "pos(" << flockData[i*6] << ", " << flockData[i*6+1] << ", " << flockData[i*6+2] << 
    		"); dir(" << flockData[i*6+3] << ", " << flockData[i*6+4] << ", " << flockData[i*6+5] << ")" << std::endl;
    }
}

void printNeighborhoods(){
    
    for(int i = 0; i < flockDim; i++){
				std::cout << i << ": ";
				for(int j = 0; j < flockDim; j++){
						if(neighborhoods[i*flockDim+j]){
								std::cout << j << ", ";
						}
				}
				std::cout << std::endl;
    }
}

void updateFlock(float time){
    
    float* cohesion = (float*) malloc(3*sizeof(float));
    float* separation = (float*) malloc(3*sizeof(float));
    float* align = (float*) malloc(3*sizeof(float));
    float* finalDirection = (float*) malloc(3*sizeof(float));
		for(int i = 0; i < flockDim; i++){
      
        getSeparationDirection(i, separation);
        getCohesionDirection(i, cohesion);
        getAlignDirection(i, align);

        blendDirections(separation, cohesion, align, finalDirection);
        if (finalDirection[0] != 0 || finalDirection[1] != 0 || finalDirection[2] != 0) {
            flockData[i*6+3] = finalDirection[0];
						flockData[i*6+4] = finalDirection[1];
						flockData[i*6+5] = finalDirection[2];
        }

        moveBoid(i, time);
    }

    free(cohesion);
    free(separation);
    free(align);
    free(finalDirection);

    computeNeighborhoods();
}

void getSeparationDirection(int i, float* separation){
    
    float* tmp = (float*) malloc(3*sizeof(float));
		float magnitude;
    for(int j = 0; j < flockDim; j++){
				if(neighborhoods[i*flockDim+j]){
						vector3Sub(flockData+i*6, flockData+j*6, tmp);
						magnitude = vector3Magnitude(tmp);
						vector3Normalize(tmp);
						vector3Mul(tmp, 1/(magnitude + 0.0001), tmp);
						vector3Sum(separation, tmp, separation);
				}
    }

    vector3Normalize(separation);
    vector3Mul(separation, separationWeigth, separation);

    free(tmp);
}

void getCohesionDirection(int i, float* cohesion){
    
    float count = 0.0;
    for(int j = 0; j < flockDim; j++){
				if(neighborhoods[i*flockDim+j]){
						vector3Sum(cohesion, flockData+j*6, cohesion);
						count++;
				}
    }

    if(count != 0){
        vector3Mul(cohesion, 1.0/count, cohesion);
        vector3Sub(cohesion, flockData+i*6, cohesion);
    }

    vector3Normalize(cohesion);
    vector3Mul(cohesion, cohesionWeigth, cohesion);
}

void getAlignDirection(int i, float* align){
    
    for(int j = 0; j < flockDim; j++){
				if(neighborhoods[i*flockDim+j]){
        		vector3Sum(align, flockData+j*6+3, align);
				}
    }

    vector3Normalize(align);
    vector3Mul(align, alignWeigth, align);
}

void moveBoid(int i, float time) { 
    
    for(int j = 0; j < 3; j++){
        flockData[i*6+j] += flockData[i*6+3+j] * velocity * time;
    }
}

Overwriting FlockingBehaviourSeq.cu


In [ ]:
%%writefile Boid.h
#include <vector>
#include <iostream>
#include <iomanip>
#include <string>

#ifndef BOID_H
#define BOID_H
class Boid{
    
    public:
    Boid();

    Boid(const int, const float, float*, float*);

    float* getDirection() const;
    float* getPosition() const;
    const int& getId() const;
    void setDirection(float*);
    void setPosition(float*);

    // move the boid by updating its position based on its current direction, velocity and time passed since the last update
    void move(float);

    void print() const;

    private:
    int id; // unique id of the boid
    float velocity; // boid velocity in meters per second
    float* position; // one unit is one meter
    float* direction; // normalized vector
};
#endif

Overwriting Boid.h


In [ ]:
%%writefile Boid.cpp
#include "Boid.h"

Boid::Boid(const int id, const float vel, float* pos, float* dir): id{id}, velocity{vel}, direction{dir}, position{pos} {}

float* Boid::getDirection() const { return direction; }

float* Boid::getPosition() const { return position; }

const int& Boid::getId() const { return id; }

void Boid::setDirection(float* dir) { 
    direction[0] = dir[0]; 
    direction[1] = dir[1]; 
    direction[2] = dir[2]; 
}

void Boid::setPosition(float* pos) { 
    position[0] = pos[0];
    position[1] = pos[1];
    position[2] = pos[2];
}

void Boid::move(float time) { 
    
    for(int i = 0; i < 3; i++){
        position[i] += direction[i] * velocity * time;
    }
}

void Boid::print() const{
    std::cout << std::setprecision(4) << "Boid " << id << ": " << "pos(" << position[0] << ", " << position[1] << ", " << position[2] << 
    "); dir(" << direction[0] << ", " << direction[1] << ", " << direction[2] << ")" << std::endl;
}

Overwriting Boid.cpp


In [ ]:
%%writefile Flock.h
#include <iostream>
#include <map>
#include <vector>
#include "Boid.h"
#include "Utils.h"

#ifndef FLOCK_H
#define FLOCK_H
class Flock{
    
    public:
    Flock();
    Flock(std::map<int, Boid>&&);

    const std::map<int, std::vector<int>>& getNeighborhoodMap() const;
    const std::map<int, Boid>& getBoidsMap() const;

    const std::vector<int>& getNeighborhood(const Boid&) const;
    void setNeighborhoodDim(const int);
    void setBlendingWeigths(const int, const int, const int);

    // returns the list of boids that are nearer than neighborhoodDim from the passed boid
    std::vector<int> computeNeighborhood(const Boid&);

    // adds a boid to the flock
    void addBoid(const Boid&);

    // computes the neighborhoods of each boid based on the current situation and puts them inside the neighborhoodMap
    void updateNeighborhoodMap();

    // for each boid computes all the components that influence its direction, blends them and set the resulting 
    // direction as the current direction of the boid only if it is not (0,0,0). otherwise the old direction is maintained
    void updateFlock(float);

    void getSeparationDirection(const int, float*) const;
    void getCohesionDirection(const int, float*) const;
    void getAlignDirection(const int, float*) const;

    void print() const;
    void printNeighborhoods() const;

    private:
    std::map<int, std::vector<int>> neighborhoodMap; // associates the neighborhood of a boid to the id of the boid
    std::map<int, Boid> boidsMap; // associates a boid to its id
    float neighborhoodDim; // dimension of the neighborhood in meters
    float separationWeigth; // weight of the separation component in the blending
    float cohesionWeigth; // weight of the cohesion component in the blending
    float alignWeigth; // weight of the align component in the blending
};
#endif

Overwriting Flock.h


In [ ]:
%%writefile Flock.cpp
#include "Flock.h"

Flock::Flock(): boidsMap{}, neighborhoodMap{} {}

Flock::Flock(std::map<int, Boid>&& boidsMap): boidsMap{boidsMap} {
    
    updateNeighborhoodMap();
}

const std::map<int, std::vector<int>>& Flock::getNeighborhoodMap() const { return neighborhoodMap; }

const std::map<int, Boid>& Flock::getBoidsMap() const { return boidsMap; }

const std::vector<int>& Flock::getNeighborhood(const Boid& b) const { return neighborhoodMap.at(b.getId()); }

void Flock::setNeighborhoodDim(int value){ neighborhoodDim = value; }

void Flock::setBlendingWeigths(const int s, const int c, const int a){
    separationWeigth = s;
    cohesionWeigth = c;
    alignWeigth = a;
}

std::vector<int> Flock::computeNeighborhood(const Boid& b){
    
    std::vector<int> neighborhood{};
    for(const auto& elem : boidsMap){
        
        if(elem.first != b.getId() && vector3Distance(elem.second.getPosition(), b.getPosition()) <= neighborhoodDim){
            neighborhood.push_back(elem.first);
        }
    }

    return neighborhood;
}

void Flock::addBoid(const Boid& b){
    boidsMap.insert(std::pair<int,Boid>(b.getId(), b));
    neighborhoodMap.insert(std::pair<int,std::vector<int>>(b.getId(), std::vector<int>{}));
}

void Flock::updateNeighborhoodMap(){
    
    for(const auto& elem : boidsMap){
        if(neighborhoodMap.find(elem.first) != neighborhoodMap.end()){
            neighborhoodMap[elem.first] = computeNeighborhood(elem.second);
        }
        else{
            neighborhoodMap.insert(std::pair<int,std::vector<int>>(elem.first, computeNeighborhood(elem.second)));
        }
    }
}

void Flock::updateFlock(float time){
    
    float* cohesion = (float*) malloc(3*sizeof(float));
    float* separation = (float*) malloc(3*sizeof(float));
    float* align = (float*) malloc(3*sizeof(float));
    float* finalDirection = (float*) malloc(3*sizeof(float));
		for(auto& elem : boidsMap){
      
        getSeparationDirection(elem.first, separation);
        getCohesionDirection(elem.first, cohesion);
        getAlignDirection(elem.first, align);

        blendDirections(separation, cohesion, align, finalDirection);
        if (finalDirection[0] != 0 || finalDirection[1] != 0 || finalDirection[2] != 0) {
            elem.second.setDirection(finalDirection);
        }

        elem.second.move(time);
    }

    free(cohesion);
    free(separation);
    free(align);
    free(finalDirection);

    updateNeighborhoodMap();
}

void Flock::getSeparationDirection(int b, float* separation) const{
    
    float* tmp = (float*) malloc(3*sizeof(float));
    for(const auto& n : neighborhoodMap.at(b)){
        vector3Sub(boidsMap.at(b).getPosition(), boidsMap.at(n).getPosition(), tmp);
        vector3Normalize(tmp);
        vector3Mul(tmp, 1/(vector3Magnitude(tmp) + 0.0001), tmp);
        vector3Sum(separation, tmp, separation);
    }

    vector3Normalize(separation);
    vector3Mul(separation, separationWeigth, separation);

    free(tmp);
}

void Flock::getCohesionDirection(int b, float* cohesion) const{
    
    float count = 0.0;
    for(const auto& n : neighborhoodMap.at(b)){
        vector3Sum(cohesion, boidsMap.at(n).getPosition(), cohesion);
        count++;
    }

    if(count != 0){
        vector3Mul(cohesion, 1.0/count, cohesion);
        vector3Sub(cohesion, boidsMap.at(b).getPosition(), cohesion);
    }

    vector3Normalize(cohesion);
    vector3Mul(cohesion, cohesionWeigth, cohesion);
}

void Flock::getAlignDirection(int b, float* align) const{
    
    for(const auto& n : neighborhoodMap.at(b)){
        vector3Sum(align, boidsMap.at(n).getDirection(), align);
    }

    vector3Normalize(align);
    vector3Mul(align, alignWeigth, align);
}

void Flock::print() const{
     
    for(const auto& elem : boidsMap){
        elem.second.print();
    }
}

void Flock::printNeighborhoods() const{
     
    for(const auto& elem : boidsMap){
        std::cout << elem.first << ": ";
        for(const auto& elem : getNeighborhood(elem.second)){
            std::cout << elem << ", ";
        }
        std::cout << std::endl;
    }
}

Overwriting Flock.cpp


In [ ]:
%%writefile Utils.h
#include <vector>
#include <math.h>
#include <chrono>
#include <sys/time.h>

#ifndef UTILS_H
#define UTILS_H

typedef unsigned long ulong;
typedef unsigned int uint;

// returns the distance between the passed vectors
float vector3Distance(const float*, const float*);

// returns the sum of the passed vectors
void vector3Sum(const float*, const float*, float*);

// returns the subtraction of the passed vectors
void vector3Sub(const float*, const float*, float*);

// returns the multiplication of the passed vectors
void vector3Mul(const float*, const float, float*);

// returns the blending of the passed vectors representing directions
void blendDirections(const float*, const float*, const float*, float*);

// returns the magnitude of the passed vector
float vector3Magnitude(const float*);

// normalizes the passed vector
void vector3Normalize(float*);

inline double seconds() {
    struct timeval tp;
    struct timezone tzp;
    int i = gettimeofday(&tp, &tzp);
    return ((double)tp.tv_sec + (double)tp.tv_usec * 1.e-6);
}

#endif

Overwriting Utils.h


In [ ]:
%%writefile Utils.cpp
#include "Utils.h"

float vector3Distance(const float* v, const float* w){
    return sqrt(pow(w[0] - v[0], 2) + pow(w[1] - v[1], 2) + pow(w[2] - v[2], 2));
}

void vector3Sum(const float* v, const float* w, float* res){
    
    res[0] = v[0] + w[0];
    res[1] = v[1] + w[1];
    res[2] = v[2] + w[2];
}

void vector3Sub(const float* v, const float* w, float* res){
    
    res[0] = v[0] - w[0];
    res[1] = v[1] - w[1];
    res[2] = v[2] - w[2];
}

void vector3Mul(const float* v, const float n, float* res){
    
    res[0] = v[0] * n;
    res[1] = v[1] * n;
    res[2] = v[2] * n;
}

void blendDirections(const float* v, const float* w, const float* u, float* res){
    
    res[0] = 0;
    res[1] = 0;
    res[2] = 0;
    vector3Sum(res, v, res);
    vector3Sum(res, w, res);
    vector3Sum(res, u, res);
}

float vector3Magnitude(const float* v){
    return sqrt(pow(v[0], 2) + pow(v[1], 2) + pow(v[2], 2));
}

void vector3Normalize(float* v){
    if(v[0] != 0 || v[1] != 0 || v[2] != 0){
        float magnitude = vector3Magnitude(v);
        v[0] *= (1/magnitude);
        v[1] *= (1/magnitude);
        v[2] *= (1/magnitude);
    }
}

Overwriting Utils.cpp


In [ ]:
# Compilation (e.g. -arch=sm_37 if compute capability is 3.7)
!nvcc -arch=sm_37 FlockingBehaviourSeq.cu Boid.cpp Flock.cpp Utils.cpp -o FlockingBehaviourSeq

# Execution
!./FlockingBehaviourSeq

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


CPU Flock generation...
    CPU elapsed time: 0.00161 (sec)
first: -17.36700
first: -1.93900
first: -33.01900
first: 0.77666
first: -0.27083
first: -0.56872
first: 7.90900
first: -27.28000
first: 21.53100
first: 0.68623
first: 0.60659
first: 0.40142
last: 0.68797
last: -0.14308
last: 0.71150
last: -18.72300
last: 49.92900
last: 0.34900
last: 0.72526
last: 0.43646
last: 0.53244
last: -19.47000
last: -22.25100
last: 6.98800


CPU neighborhoods generation...
    CPU elapsed time: 3.52921 (sec)


CPU Flock update...
    CPU elapsed time: 17.29297 (sec)

# Flocking Behaviour Parallel

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
%cd "content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel"
%ls

/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel
deviceQuery          FlockingBehaviourPar      helper.h   Utils.h
deviceQuery.cu       FlockingBehaviourPar.cu   src/
FlockingBehaviour.h  FlockingBehaviourSeq.cpp  Utils.cpp


In [2]:
#%cd ..
#%cd "FlockingBehaviourSequential"
%ls

drive/  sample_data/


In [3]:
%cd ..
%cd "FlockingBehaviourParallel"
%ls

/
[Errno 2] No such file or directory: 'FlockingBehaviourParallel'
/
bin/      dev/   lib32/  opt/         run/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  proc/        sbin/  tmp/
content/  home/  media/  python-apt/  srv/   tools/
datalab/  lib/   mnt/    root/        sys/   usr/


In [ ]:
%%writefile FlockingBehaviour.h
#include "Utils.h"
#include <stdlib.h>
#include <time.h> 
#include <iostream> 
#include <iomanip>
#include <curand_kernel.h>

#ifndef FLOCKING_H
#define FLOCKING_H

extern float velocity;
extern double updateTime;
extern float separationWeight;
extern float cohesionWeight;
extern float alignWeight;
extern int flockDim;
extern float neighDim;
extern float tolerance;
extern int minRand;
extern int maxRand;
extern float decimals;
extern int iterations;
extern int generationsPerThread;

extern float* flockData;
extern bool* neighborhoods;
extern bool* neighborhoodsSeq;

void generateFlock(float*, int, int, int, int);

void computeNeighborhoods(bool*, float*, int, float);
bool checkNeighborhoodsCorrectness(bool*, bool*, float*, int);

void updateFlock(float, bool*, float*, float*, int, int, float, float, float);
void getSeparationDirection(int, float*, bool*, float*, int, float*);
void getCohesionDirection(int, float*, bool*, float*, int, float*);
void getAlignDirection(int, float*, bool*, float*, int, float*);
void moveBoid(int, float, float*, float*, int);
bool checkUpdateCorrectness(float*, float*, int);

void printNeighborhoods(bool*, int);
void printFlock(float*, int);
void printBoid(int, float*, int);
#endif

Overwriting FlockingBehaviour.h


In [ ]:
%%writefile FlockingBehaviourSeq.cpp
#include "FlockingBehaviour.h"

/*
 * Host function that computes distances between boids and fills the boolean matrix representing the neighborhoods
 * For future correctness check needs it uses as input the same data that will be used by the GPU (flockData and neighborhoods), but saves the results in the flockDataSeq array
 */
void computeNeighborhoods(bool* neighborhoodsSeq, float* flockData, int flockDim, float neighDim){

    float dist;
		for(int i = 0; i < flockDim; i++){
				for(int j = 0; j < flockDim; j++){
						
						//if the current cell is the upper right triangle of the matrix calculate the value
						if(j > i){	
								dist = vector3Distance(flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim);
								neighborhoodsSeq[i*flockDim+j] = (dist < neighDim);
						}
						else if(i == j){

								//if the current cell is on the diagonal the value is always zero because a boid is not neighbor of themself
								neighborhoodsSeq[i*flockDim+j] = 0;
						}
						else{
								
								//if the current cell is the lower left triangle of the matrix copy the corresponding value from the upper right triangle as the matrix is symmetric
								neighborhoodsSeq[i*flockDim+j] = neighborhoodsSeq[j*flockDim+i];
						}
				}
    }
}

/*
 * Device function for the computation of the separation component of one boid: the component is the normalized average repulsion vector from the neighbours
 */
void getSeparationDirection(int i, float* separation, bool* neighborhoodsSeq, float* flockData, int flockDim, float separationWeight){
    
		separation[0] = 0;
		separation[1] = 0;
		separation[2] = 0;

		//compute the average repulsion vector by summing the repulsion vectors
    float* tmp = (float*) malloc(3*sizeof(float));
		float magnitude;
    for(int j = 0; j < flockDim; j++){
				if(neighborhoodsSeq[i*flockDim+j]){
						
						//calculate the vector from the boid position to the neighbour position
						vector3Sub(flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim, tmp);

						//normalize it and divide it by its magnitude to obtain the repulsion vector
						magnitude = vector3Magnitude(tmp);
						vector3Normalize(tmp);
						vector3Mul(tmp, 1/(magnitude + 0.0001), tmp);

						//sum it to the current separation
						vector3Sum(separation, tmp, separation);
				}
    }

		//normalize and weight the component
    vector3Normalize(separation);
    vector3Mul(separation, separationWeight, separation);

    free(tmp);
}

/*
 * Host function for the computation of the cohesion component of one boid: the component is the normalized vector from the boid current position to the average position of its neighbours
 */
void getCohesionDirection(int i, float* cohesion, bool* neighborhoodsSeq, float* flockData, int flockDim, float cohesionWeight){
    
		cohesion[0] = 0;
		cohesion[1] = 0;
		cohesion[2] = 0;

		//compute the sum of all the neighbours positions
    float count = 0.0;
    for(int j = 0; j < flockDim; j++){
				if(neighborhoodsSeq[i*flockDim+j]){
						vector3Sum(cohesion, cohesion+1, cohesion+2, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim, cohesion);
						count++;
				}
    }

		//calculate the average position and the vector to it only if there is at least one neighbours, otherwise the cohesion component remains the zero vector
    if(count != 0){
        vector3Mul(cohesion, 1.0/count, cohesion);
        vector3Sub(cohesion, cohesion+1, cohesion+2, flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, cohesion);
    }

		//normalize and weight the component
    vector3Normalize(cohesion);
    vector3Mul(cohesion, cohesionWeight, cohesion);
}

/*
 * Host function for the computation of the align component of one boid: the component is the average direction of the neighbours
 */
void getAlignDirection(int i, float* align, bool* neighborhoodsSeq, float* flockData, int flockDim, float alignWeight){
		
		align[0] = 0;
		align[1] = 0;
		align[2] = 0;
    
		//compute the average direction by summing the neighbours directions
    for(int j = 0; j < flockDim; j++){
				if(neighborhoodsSeq[i*flockDim+j]){
        		vector3Sum(align, align+1, align+2, flockData+j+3*flockDim, flockData+j+4*flockDim, flockData+j+5*flockDim, align);
				}
    }

		//normalize and weight the component
    vector3Normalize(align);
    vector3Mul(align, alignWeight, align);
}

/*
 * Host function that updates the boid position by moving it towards its direction at the given velocity
 * For future correctness check needs it uses as input the same data that will be used by the GPU (flockData and neighborhoods), but saves the results in the flockDataSeq array
 */
void moveBoid(int i, float time, float* flockData, float* flockDataSeq, int flockDim) { 

    for(int j = 0; j < 3; j++){
        flockDataSeq[i+j*flockDim] = flockData[i+j*flockDim] + flockDataSeq[i+(j+3)*flockDim] * velocity * time;
    }
}

/*
 * Host function that determines the new direction of each boid based on its neighbors status and then determines each boid new position moving in the new direction at the given velocity.
 * For future correctness check needs it uses as input the same data that will be used by the GPU (flockData and neighborhoods), but saves the results in the flockDataSeq array
 */
void updateFlock(float time, bool* neighborhoodsSeq, float* flockData, float* flockDataSeq, int flockDim, int neighDim, float separationWeight, float cohesionWeight, float alignWeight){
    
    float* cohesion = (float*) malloc(3*sizeof(float));
    float* separation = (float*) malloc(3*sizeof(float));
    float* align = (float*) malloc(3*sizeof(float));
    float* finalDirection = (float*) malloc(3*sizeof(float));
		for(int i = 0; i < flockDim; i++){
      
				//calculate the components of the new direction of the boid
        getSeparationDirection(i, separation, neighborhoodsSeq, flockData, flockDim, separationWeight);
        getCohesionDirection(i, cohesion, neighborhoodsSeq, flockData, flockDim, cohesionWeight);
        getAlignDirection(i, align, neighborhoodsSeq, flockData, flockDim, alignWeight);

				//blend them togheter and update the direction of the boid if the resulting direction is not the zero vector
        blendDirections(separation, cohesion, align, finalDirection);
        if (finalDirection[0] != 0 || finalDirection[1] != 0 || finalDirection[2] != 0) {
            flockDataSeq[i+3*flockDim] = finalDirection[0];
						flockDataSeq[i+4*flockDim] = finalDirection[1];
						flockDataSeq[i+5*flockDim] = finalDirection[2];
        }
				else{
						
						//otherwise keep the current direction

						flockDataSeq[i+3*flockDim] = flockData[i+3*flockDim];
						flockDataSeq[i+4*flockDim] = flockData[i+4*flockDim];
						flockDataSeq[i+5*flockDim] = flockData[i+5*flockDim];
				}

				//update the position
        moveBoid(i, time, flockData, flockDataSeq, flockDim);
    }

    free(cohesion);
    free(separation);
    free(align);
    free(finalDirection);

		//computeNeighborhoods(neighborhoodsSeq, flockData, flockDim, neighDim);
}

/*
 * Host function that generates the positions and the directions of all boids in the flock
 */
void generateFlock(float* flockDataSeq, int numsToGenerate, int maxRand, int minRand, int div){
    
    for(int i = 0; i < numsToGenerate; i+=6){
			flockDataSeq[i] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+1] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+2] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+3] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+4] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+5] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
		
			//normalize the direction
			vector3Normalize(flockDataSeq+i+3);
		}
}

/*
 * Host function that prints a single boid of the flock
 */
void printBoid(int i, float* flockData, int flockDim){
		
		std::cout << std::setprecision(4) << "Boid " << i << ": " << "pos(" << flockData[i] << ", " << flockData[i+flockDim] << ", " << flockData[i+flockDim*2] << 
		"); dir(" << flockData[i+flockDim*3] << ", " << flockData[i+flockDim*4] << ", " << flockData[i+flockDim*5] << ")" << std::endl;
    printf("Boid direction magnitude: %.5f\n", vector3Magnitude(flockData+i+flockDim*3, flockData+i+flockDim*4, flockData+i+flockDim*5));
}

/*
 * Host function that prints all the boids in the flock
 */
void printFlock(float* flockData, int flockDim){
    
    for(int i = 0; i < flockDim; i++){
				printBoid(i, flockData, flockDim);
    }
}

/*
 * Host function that prints all the neighbors of each boid
 */
void printNeighborhoods(bool* neighborhoodsSeq, int flockDim){
    
    for(int i = 0; i < flockDim; i++){
				std::cout << i << ": ";
				for(int j = 0; j < flockDim; j++){
						if(neighborhoods[i*flockDim+j]){
								std::cout << j << ", ";
						}
				}
				std::cout << std::endl;
    }
}

/*
 * Host function for checking if the results of the neighborhoods computation made by the GPU and by the CPU is the same
 */
bool checkNeighborhoodsCorrectness(bool* neighborhoods, bool* neighborhoodsSeq, float* flockData, int flockDim){
    
    bool correct = 1;
		for(int i = 0; i < flockDim; i++){
				
				if(!correct){
						break;
				}

				for(int j = 0; j < flockDim; j++){

						//compare each value for which the distance from the boids was different from the neighbour dimension (in that cases the distance comparison may be have a different outcomes for
						//GPU and for CPU because of imprecision in the float comparison)
						if(neighborhoodsSeq[i*flockDim+j] != neighborhoods[i*flockDim+j] && 
						   vector3Distance(flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim) != neighDim){
								
								correct = 0;

								//print debug data if they are not equal
                printf("i: %i --- j: %i\n", i,j);
								printf("seq: %i, %.5f --- par: %i, %.5f\n", neighborhoodsSeq[i*flockDim+j], vector3Distance(flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim),
								        neighborhoods[i*flockDim+j], sqrt(pow(flockData[i] - flockData[j], 2) + pow(flockData[i+flockDim] - flockData[j+flockDim], 2) + pow(flockData[i+2*flockDim] - flockData[j+2*flockDim], 2)));
								
								printBoid(i, flockData, flockDim);
								printBoid(j, flockData, flockDim);

								break;
						}
				}
    }

    return correct;
}

/*
 * Host function for checking if the results of the update made by the GPU and by the CPU is the same
 */
bool checkUpdateCorrectness(float* flockData, float* flockDataSeq, int flockDim){
		
		bool correct = 1;
		for(int i = 0; i < flockDim * 6; i++){

				//compare the float values with a very small tolerance
				if(fabs(flockData[i] - flockDataSeq[i]) > tolerance){
						
					  correct = 0;

						//print debug data if they are not equal
						printf("i: %i\n", i);
						printf("seq: %.5f --- par: %.5f\n", flockDataSeq[i], flockData[i]);

						break;
				}
    }

    return correct;
}

Overwriting FlockingBehaviourSeq.cpp


In [ ]:
%%writefile FlockingBehaviourPar.cu
#include "FlockingBehaviour.h"

#define GEN_BLOCK_SIZE 128
#define GEN_GRID_SIZE 16
//Config: fd->40960  gen->20  b_s->128  g_s->16  -----  gen->1  b_s->128  g_s->320 (higher occupancy, but less speed-up)
//Config: fd->102400  gen->50  b_s->128  g_s->16
//Config: fd->10240000  gen->2500  b_s->128  g_s->32
//Config: fd->102400000  gen->5000  b_s->320  g_s->64  -----  gen->2500  b_s->128  g_s->160 (higher occupancy, but less speed-up)

#define NEIGH_BLOCK_SIZE 1024

#define DIRECTION_BLOCK_SIZE 192 
//must be divisible by 3 nd 32

#define UPDATE_BLOCK_SIZE 128 

float velocity = 20; // boid velocity in meters per second
double updateTime = 1.5; // update time of the simulation in seconds
float separationWeight = 1; // weight of the separation component in the blending
float cohesionWeight = 1; // weight of the cohesion component in the blending
float alignWeight = 1; // weight of the align component in the blending
int flockDim = 40960; // number of boids in the flock
float neighDim = 50; // dimension of the neighborhood in meters
float tolerance = 0.001f; //tolerance for float comparison
int minRand = -50000; // minimum value that can be generated for initial position and direction
int maxRand = 50000; // maximum value that can be generated for initial position and direction
float decimals = 3; // number of decimal digits in the generated values for initial position and direction
int iterations = 1; // number of updates
int generationsPerThread = 20; // number of boids a thread must generate

float* flockData;
float* flockDataSeq;
bool* neighborhoods;
bool* neighborhoodsSeq;
float* tmp;

__constant__ float movementDev;
__constant__ float separationWeightDev;
__constant__ float cohesionWeightDev;
__constant__ float alignWeightDev;
__constant__ int flockDimDev;
__constant__ float neighDimDev;
__constant__ int unitNumDev;
__constant__ float toleranceDev;
__constant__ int minRandDev;
__constant__ int minMaxDiffDev;
__constant__ float divDev;
__constant__ int threadsNumDev;
__constant__ int generationsPerThreadDev;
__constant__ int lastThreadGenerationsDev;

/*
 * Kernel for cuRAND states initialization
 */
__global__ void initializeStates(uint seed, curandState* states) {

		uint tid = threadIdx.x + blockDim.x * blockIdx.x;

		curand_init(seed, tid, 0, &states[tid]);
}

/*
 * Kernel for flock generation: generates boids with random initial position and direction
 */
__global__ void generateBoidsStatus(float* generated, curandState* states) {

	uint tid = threadIdx.x + blockDim.x * blockIdx.x;

	if(tid < threadsNumDev){
			
			curandState localState = states[tid];

			//to avoid accesses out of bounds
			//if the thread is the last make it generate the numbers for the remaining boids, otherwise make it generate generationsPerThread numbers
			int myGenerations;
			if(tid == (threadsNumDev - 1)){
					myGenerations = lastThreadGenerationsDev;
			}
			else{
					myGenerations = generationsPerThreadDev;
			}
	
			float d1, d2, d3;
			float value;
			uint pos = tid - threadsNumDev;
			for(uint i = 0; i < myGenerations; i++){
					
					//generate the boid direction
					d1 = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;
					d2 = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;
					d3 = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;

					value = sqrt(d1*d1 + d2*d2 + d3*d3);

					//if the magnitude of the direction is 0 the value must be 1 to avoid dividing for 0, otherwise calculate the value to normalize it as 1/magnitude
					value = !(value == 0) * 1/(value + (value == 0));

					//if the last thread finished its generations we have to skip one position less
					if(i > lastThreadGenerationsDev){
							pos += (threadsNumDev - 1);
					}
					else{
							pos += threadsNumDev;
					}

					//generate and save the boid position
					generated[pos] = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;
					generated[pos + flockDimDev] = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;
					generated[pos + 2 * flockDimDev] = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;

					//normalize and save boid direction
					generated[pos + 3 * flockDimDev] = d1 * value;
					generated[pos + 4 * flockDimDev] = d2 * value;
					generated[pos + 5 * flockDimDev] = d3 * value;
			}		
			
			states[tid] = localState;
	}
}

/*
 * Kernel for neighborhoods computation: computes distances and fills the boolean matrix
 */
__global__ void computeAllNeighborhoods(float* flockData, bool* neighborhoods) {

		uint tid = threadIdx.x + blockDim.x * blockIdx.x;

		if(tid < threadsNumDev){

				uint i = (tid/flockDimDev) * flockDimDev/unitNumDev;
				uint max = i + flockDimDev/unitNumDev;
				tid = tid % flockDimDev;

				if(max > flockDimDev){
						printf("error i: %i\n",i);
				}
				if(tid >= flockDimDev){
						printf("error tid: %i\n",tid);
				}

				bool value;
				for(; i < max; i++){

						if(tid == i){
								neighborhoods[i*flockDimDev+tid] = 0;
						}
						else{
								value = sqrt(pow(flockData[tid] - flockData[i], 2) + pow(flockData[tid+flockDimDev] - flockData[i+flockDimDev], 2) + pow(flockData[tid+2*flockDimDev] - flockData[i+2*flockDimDev], 2)) <= neighDimDev;
								neighborhoods[i*flockDimDev+tid] = value;
						}
				}
		}
}

/*
 * Device function for the computation of the cohesion component of one boid: the component is the normalized vector from the boid current position to the average position of its neighbours
 */
__device__ void computeCohesionGPU(uint boidId, uint sharedOffset, uint unitSize, bool* neighborhoods, float* flockData, float* cohesions){
		
		cohesions[sharedOffset] = 0;
		cohesions[sharedOffset+unitSize] = 0;
		cohesions[sharedOffset+2*unitSize] = 0;

		//compute the sum of all the neighbours positions
		float count = 0.0;
		for(int i = 0; i < flockDimDev; i++){
				
				if(neighborhoods[i*flockDimDev+boidId]){
						
						cohesions[sharedOffset] += flockData[i];
						cohesions[sharedOffset+unitSize] += flockData[i+flockDimDev];
						cohesions[sharedOffset+2*unitSize] += flockData[i+2*flockDimDev];
						count += 1;
				}
		}

		//calculate the average position and the vector to it only if there is at least one neighbours, otherwise the cohesion component remains the zero vector
		if(count != 0){
				
				count = 1.0/count;
				cohesions[sharedOffset] *= count;
				cohesions[sharedOffset+unitSize] *= count;
				cohesions[sharedOffset+2*unitSize] *= count;

				cohesions[sharedOffset] -= flockData[boidId];
				cohesions[sharedOffset+unitSize] -= flockData[boidId+flockDimDev];
				cohesions[sharedOffset+2*unitSize] -= flockData[boidId+2*flockDimDev];
		}

		//normalize and weight the component
		float normValue;
		normValue = sqrt(cohesions[sharedOffset]*cohesions[sharedOffset] + cohesions[sharedOffset+unitSize]*cohesions[sharedOffset+unitSize] + cohesions[sharedOffset+2*unitSize]*cohesions[sharedOffset+2*unitSize]);
		normValue = !(normValue == 0) * 1/(normValue + (normValue == 0));
		normValue *= cohesionWeightDev;

		cohesions[sharedOffset] *= normValue;
		cohesions[sharedOffset+unitSize] *= normValue;
		cohesions[sharedOffset+2*unitSize] *= normValue;
}

/*
 * Device function for the computation of the separation component of one boid: the component is the normalized average repulsion vector from the neighbours
 */
__device__ void computeSeparationGPU(uint boidId, uint sharedOffset, uint unitSize, bool* neighborhoods, float* flockData, float* separations){
		
		separations[sharedOffset] = 0;
		separations[sharedOffset+unitSize] = 0;
		separations[sharedOffset+2*unitSize] = 0;

		//compute the average repulsion vector by summing the repulsion vectors
		float tmp1;
		float tmp2;
		float tmp3;
		float magValue;
		float normValue;
		for(int i = 0; i < flockDimDev; i++){
				
				if(neighborhoods[i*flockDimDev+boidId]){
						
						//calculate the vector from the boid position to the neighbour position
						tmp1 = flockData[boidId] - flockData[i];
						tmp2 = flockData[boidId+flockDimDev] - flockData[i+flockDimDev];
						tmp3 = flockData[boidId+2*flockDimDev] - flockData[i+2*flockDimDev];

						//normalize it and divide it by its magnitude to obtain the repulsion vector
						normValue = sqrt(tmp1*tmp1 + tmp2*tmp2 + tmp3*tmp3);
						magValue = normValue;
						magValue = 1/(magValue + 0.0001);
						normValue = !(normValue == 0) * 1/(normValue + (normValue == 0));
						normValue *= magValue;

						tmp1 *= normValue;
						tmp2 *= normValue;
						tmp3 *= normValue;

						//sum it to the current separation
						separations[sharedOffset] += tmp1;
						separations[sharedOffset+unitSize] += tmp2;
						separations[sharedOffset+2*unitSize] += tmp3;
				}
		}

		//normalize and weight the component
		normValue = sqrt(separations[sharedOffset]*separations[sharedOffset] + separations[sharedOffset+unitSize]*separations[sharedOffset+unitSize] + separations[sharedOffset+2*unitSize]*separations[sharedOffset+2*unitSize]);
		normValue = !(normValue == 0) * 1/(normValue + (normValue == 0));
		normValue *= separationWeightDev;

		separations[sharedOffset] *= normValue;
		separations[sharedOffset+unitSize] *= normValue;
		separations[sharedOffset+2*unitSize] *= normValue;
}

/*
 * Device function for the computation of the align component of one boid: the component is the average direction of the neighbours
 */
__device__ void computeAlignGPU(uint boidId, uint sharedOffset, uint unitSize, bool* neighborhoods, float* flockData, float* aligns){
		
		aligns[sharedOffset] = 0;
		aligns[sharedOffset+unitSize] = 0;
		aligns[sharedOffset+2*unitSize] = 0;

		//compute the average direction by summing the neighbours directions
		for(int i = 0; i < flockDimDev; i++){
				
				if(neighborhoods[i*flockDimDev+boidId]){
						
						aligns[sharedOffset] += flockData[i+3*flockDimDev];
						aligns[sharedOffset+unitSize] += flockData[i+4*flockDimDev];
						aligns[sharedOffset+2*unitSize] += flockData[i+5*flockDimDev];
				}
		}

		//normalize and weight the component
		float normValue;
		normValue = sqrt(aligns[sharedOffset]*aligns[sharedOffset] + aligns[sharedOffset+unitSize]*aligns[sharedOffset+unitSize] + aligns[sharedOffset+2*unitSize]*aligns[sharedOffset+2*unitSize]);
		normValue = !(normValue == 0) * 1/(normValue + (normValue == 0));
		normValue *= alignWeightDev;

		aligns[sharedOffset] *= normValue;
		aligns[sharedOffset+unitSize] *= normValue;
		aligns[sharedOffset+2*unitSize] *= normValue;
}

/*
 * Kernel for computing the new direction: determines the new direction of each boid based on its neighbors status
 */
__global__ void computeDirection(float* flockData, bool* neighborhoods, float* tmp, uint unitSize) {

		uint sharedOffset = threadIdx.x % unitSize;
		uint boidId = unitSize * blockIdx.x + sharedOffset;

		if(boidId < flockDimDev){
				
				extern __shared__ float cohesions[];
				float* separations = cohesions+3*unitSize;
				float* aligns = cohesions+6*unitSize;
				
				if(threadIdx.x < unitSize){
						
						// first unit calculates cohesion

						computeCohesionGPU(boidId, sharedOffset, unitSize, neighborhoods, flockData, cohesions);
				}
				else if(threadIdx.x >= unitSize && threadIdx.x < 2*unitSize){
						
						// second unit calculates separation

						computeSeparationGPU(boidId, sharedOffset, unitSize, neighborhoods, flockData, separations);
				}
				else{
						
						// third unit calculates align

						computeAlignGPU(boidId, sharedOffset, unitSize, neighborhoods, flockData, aligns);
			  }

				__syncthreads();

				// blend contributions and normalize them

				float tmp1;
				float tmp2;
				float tmp3;
				if(threadIdx.x < unitSize){
						
						tmp1 = cohesions[sharedOffset] + separations[sharedOffset] + aligns[sharedOffset];
						tmp2 = cohesions[sharedOffset+unitSize] + separations[sharedOffset+unitSize] + aligns[sharedOffset+unitSize];
						tmp3 = cohesions[sharedOffset+2*unitSize] + separations[sharedOffset+2*unitSize] + aligns[sharedOffset+2*unitSize];

						float normValue;
						normValue = sqrt(tmp1*tmp1 + tmp2*tmp2 + tmp3*tmp3);
						normValue = !(normValue == 0) * 1/(normValue + (normValue == 0));

						tmp1 *= normValue;
						tmp2 *= normValue;
						tmp3 *= normValue;

						tmp[boidId] = tmp1;
						tmp[boidId+flockDimDev] = tmp2;
						tmp[boidId+2*flockDimDev] = tmp3;
				}
		}
}

/*
 * Kernel for flock update: determines each boid new position moving in the new direction at the given velocity 
 */
__global__ void updateFlock(float* flockData, float* tmp) {

		uint tid = threadIdx.x + blockDim.x * blockIdx.x;

		if(tid < threadsNumDev){

				//update the direction only if the new direction is not the zero vector
				if(tmp[tid] != 0 || tmp[tid+flockDimDev] != 0 || tmp[tid+2*flockDimDev] != 0){
						flockData[tid+3*flockDimDev] = tmp[tid];
						flockData[tid+4*flockDimDev] = tmp[tid+flockDimDev];
						flockData[tid+5*flockDimDev] = tmp[tid+2*flockDimDev];
				}

				//move in the saved direction
				flockData[tid] += flockData[tid+3*flockDimDev] * movementDev;
				flockData[tid+flockDimDev] += flockData[tid+4*flockDimDev] * movementDev;
				flockData[tid+2*flockDimDev] += flockData[tid+5*flockDimDev] * movementDev;
		}
}

int main(void) {

		device_name();

		printf("\nFlock dimension: %i\n", flockDim);
		printf("Neighborhood dimension: %.2f\n", neighDim);
		printf("Velocity: %.2f\n", velocity);
		printf("Update time: %.2f\n", updateTime);
		printf("Iterations: %i\n", iterations);
		printf("Separation weight: %.2f\n", separationWeight);
		printf("Cohesion weight: %.2f\n", cohesionWeight);
		printf("Align weight: %.2f\n", alignWeight);
		printf("Minimum random number: %i\n", minRand);
		printf("Maximum random number: %i\n", maxRand);
		printf("Decimal digits of random numbers: %.2f\n", decimals);

		// create events to measure time
		cudaEvent_t start, stop;
		cudaEventCreate(&start);
		cudaEventCreate(&stop);
		float milliseconds = 0;
		double cpuTimeStart;
		double cpuTime;

		// -------------------------------------FLOCK GENERATION---------------------------------------------

		// prepare for flock generation
		int minMaxDiff = maxRand - minRand;
		float div = pow(10.0, decimals);
		int numsToGenerate = flockDim * 6;

		cudaFuncSetCacheConfig(generateBoidsStatus, cudaFuncCachePreferL1);
		cudaFuncSetCacheConfig(initializeStates, cudaFuncCachePreferL1);

		float* flockData;
		curandState* states;
		int blockSize = GEN_BLOCK_SIZE;
		int gridSize = GEN_GRID_SIZE; 
		CHECK(cudaMallocManaged((void **) &flockData, numsToGenerate * sizeof(float)));
		CHECK(cudaMalloc((void **) &states, blockSize * gridSize * sizeof(curandState)));
		int threadsNum = (flockDim + generationsPerThread - 1) / generationsPerThread;
		int lastThreadGenerations = flockDim - generationsPerThread * (threadsNum - 1);

		// initialize all constant values
		cudaMemcpyToSymbol(separationWeightDev, &separationWeight, sizeof(separationWeightDev));
		cudaMemcpyToSymbol(cohesionWeightDev, &cohesionWeight, sizeof(cohesionWeightDev));
		cudaMemcpyToSymbol(alignWeightDev, &alignWeight, sizeof(alignWeightDev));
		cudaMemcpyToSymbol(flockDimDev, &flockDim, sizeof(flockDimDev));
		cudaMemcpyToSymbol(neighDimDev, &neighDim, sizeof(neighDimDev));
		cudaMemcpyToSymbol(toleranceDev, &tolerance, sizeof(toleranceDev));
		cudaMemcpyToSymbol(minRandDev, &minRand, sizeof(minRandDev));
		cudaMemcpyToSymbol(minMaxDiffDev, &minMaxDiff, sizeof(minMaxDiffDev));
		cudaMemcpyToSymbol(divDev, &div, sizeof(divDev));
		cudaMemcpyToSymbol(threadsNumDev, &threadsNum, sizeof(threadsNumDev));
		cudaMemcpyToSymbol(generationsPerThreadDev, &generationsPerThread, sizeof(generationsPerThreadDev));
		cudaMemcpyToSymbol(lastThreadGenerationsDev, &lastThreadGenerations, sizeof(lastThreadGenerationsDev));

		printf("\nNeeded threads number: %i\n", threadsNum);
		printf("Threads used: %i\n", blockSize * gridSize);
		printf("Block size: %i\n", blockSize);
		printf("Grid size: %i\n",gridSize);
		printf("Generations per thread: %i\n", generationsPerThread);
		printf("Generations of last thread: %i\n", lastThreadGenerations);
		if(threadsNum > blockSize * gridSize){
				std::cout << "\nNot enough threads" << std::endl;
		}

		// generate flock
		printf("\n\nGPU Flock generation...\n");
		cudaEventRecord(start);

		initializeStates<<<gridSize, blockSize>>>(time(NULL), states);

		CHECK(cudaDeviceSynchronize());

		generateBoidsStatus<<<gridSize, blockSize>>>(flockData, states);

		cudaEventRecord(stop);
		CHECK(cudaEventSynchronize(stop));
		cudaEventElapsedTime(&milliseconds, start, stop);
		printf("    GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);

		//print some boids to check the generation correctness 
		for(int i = 0; i < 10; i++){
				printBoid(i, flockData, flockDim);
		}
		for(int i = flockDim/2-5; i < flockDim/2+5; i++){
				printBoid(i, flockData, flockDim);
		}
		for(int i = flockDim-11; i < flockDim-1; i++){
				printBoid(i, flockData, flockDim);
		}


		// generate flock sequentially to measure the speed-up
		flockDataSeq = (float*) malloc(numsToGenerate * sizeof(float));

		printf("\n\nCPU Flock generation...\n");
		cpuTimeStart = seconds();

		generateFlock(flockDataSeq, numsToGenerate, maxRand, minRand, div);

		cpuTime = seconds() - cpuTimeStart;
		printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

		printf("				Speedup: %.2f\n", cpuTime/(milliseconds / 1000));
		

		CHECK(cudaFree(states));

		// -------------------------------------NEIGHBORHOODS CALCULATION---------------------------------------------

		// prepare for neighborhood calculation
		// neighborhoods data stored in a boolean matrix
		CHECK(cudaMallocManaged((void **) &neighborhoods, flockDim * flockDim * sizeof(bool)));

		cudaFuncSetCacheConfig(computeAllNeighborhoods, cudaFuncCachePreferL1);

		int unitNum = 16; 
		int neighThreadsNum = flockDim*unitNum; 
		int neighBlockSize = NEIGH_BLOCK_SIZE;
		int neighGridSize = (neighThreadsNum + neighBlockSize - 1)/neighBlockSize;

		printf("\nNeeded threads number: %i\n", neighThreadsNum);
		printf("Threads used: %i\n", neighBlockSize * neighGridSize);
		printf("Block size: %i\n", neighBlockSize);
		printf("Grid size: %i\n", neighGridSize);
		printf("Number of units: %i\n", unitNum);
		if(neighThreadsNum > neighBlockSize * neighGridSize){
				std::cout << "\nNot enough threads" << std::endl;
		}

		cudaMemcpyToSymbol(unitNumDev, &unitNum, sizeof(unitNumDev));

		// update total threads number in constant memory
		cudaMemcpyToSymbol(threadsNumDev, &neighThreadsNum, sizeof(threadsNumDev));

		// compute all neighborhoods
		printf("\n\nGPU Neighborhoods computation...\n");
		cudaEventRecord(start);

		computeAllNeighborhoods<<<neighGridSize, neighBlockSize>>>(flockData, neighborhoods);

		cudaEventRecord(stop);
		CHECK(cudaEventSynchronize(stop));
		cudaEventElapsedTime(&milliseconds, start, stop);
		printf("    GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);

		for(int i = 0; i < 12; i++){
				printf("Element %i of neighborhoods: %d\n", i, neighborhoods[i]);
		}
		for(int i = 0; i < 12; i++){
				printf("Element %i of neighborhoods: %d\n", i, neighborhoods[(flockDim-1)*flockDim+flockDim-1-i]);
		}

		//printFlock(flockData, flockDim);		
		//std::cout << std::endl;
		//printNeighborhoods(neighborhoodsSeq, flockDim);
		//std::cout << std::endl;
		//std::cout << std::endl;


		// calculate neighborhoods sequentially to check the result and measure the speed-up
		
		neighborhoodsSeq = (bool*) malloc(flockDim * flockDim * sizeof(bool));

		printf("\n\nCPU Neighborhoods computation...\n");
		cpuTimeStart = seconds();

		computeNeighborhoods(neighborhoodsSeq, flockData, flockDim, neighDim);

		cpuTime = seconds() - cpuTimeStart;
		printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

		printf("				Speedup: %.2f\n", cpuTime/(milliseconds / 1000));
		
		printf("\nNeighborhoods computation correctness: %i\n\n", checkNeighborhoodsCorrectness(neighborhoods, neighborhoodsSeq, flockData, flockDim));
		

		//printNeighborhoods(neighborhoods, flockDim);
		//printNeighborhoods(neighborhoodsSeq, flockDim);

		// -------------------------------------FLOCK UPDATE---------------------------------------------

		//prepare for flock updates
		CHECK(cudaMallocManaged((void **) &tmp, flockDim * 3));
		float movement = velocity * updateTime;
		cudaMemcpyToSymbol(movementDev, &movement, sizeof(movementDev));

		//priviledge shared memory if a high amount is needed
		//cudaFuncSetCacheConfig(updateFlock, cudaFuncCachePreferShared);
		//cudaFuncSetCacheConfig(computeDirection, cudaFuncCachePreferShared);

		int dirBlockSize = DIRECTION_BLOCK_SIZE;
		int unitSize = dirBlockSize/3;
		int dirGridSize = (flockDim + unitSize - 1)/unitSize;

		int updateThreadsNum = flockDim;
		int updateBlockSize = UPDATE_BLOCK_SIZE;
		int updateGridSize = (updateThreadsNum + updateBlockSize - 1)/updateBlockSize;

		//prepare for CPU flock update
		float* cohesion = (float*) malloc(3*sizeof(float));
		float* separation = (float*) malloc(3*sizeof(float));
		float* align = (float*) malloc(3*sizeof(float));
		float* finalDirection = (float*) malloc(3*sizeof(float));

		printf("\nNeeded threads number: %i\n", dirBlockSize * dirGridSize);
		printf("Threads used: %i\n", dirBlockSize * dirGridSize);
		printf("Block size: %i\n", dirBlockSize);
		printf("Grid size: %i\n", dirGridSize);
		printf("Unit size: %i\n", dirBlockSize/3);

		printf("\nNeeded threads number: %i\n", updateThreadsNum);
		printf("Threads used: %i\n", updateBlockSize * updateGridSize);
		printf("Block size: %i\n", updateBlockSize);
		printf("Grid size: %i\n", updateGridSize);
		if(updateThreadsNum > updateBlockSize * updateGridSize){
				std::cout << "\nNot enough threads" << std::endl;
		}

		// start simulation loop that updates the flock each updateTime
		double loopStart = seconds();
		double tmpTime = updateTime;
		while(iterations > 0){

				auto duration = seconds() - loopStart;
				if(duration >= tmpTime)
				{		

						// update the flock sequentially to check the result and measure the speed-up
						printf("\n\nCPU Flock update...\n");
						double cpuTimeStart = seconds();

						updateFlock(updateTime, neighborhoods, flockData, flockDataSeq, flockDim, neighDim, separationWeight, cohesionWeight, alignWeight);

						double cpuTime = seconds() - cpuTimeStart;
						printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);
					  
					
						// update total threads number in constant memory
						//cudaMemcpyToSymbol(threadsNumDev, &dirThreadsNum, sizeof(threadsNumDev));
					
						// update the flock
						printf("\n\nGPU Flock update...\n");
						cudaEventRecord(start);

						computeDirection<<<dirGridSize, dirBlockSize, 3 * 3 * unitSize * sizeof(float)>>>(flockData, neighborhoods, tmp, unitSize);
					
						cudaDeviceSynchronize();
					
						// update total threads number in constant memory
						cudaMemcpyToSymbol(threadsNumDev, &updateThreadsNum, sizeof(threadsNumDev));
			
						updateFlock<<<updateGridSize, updateBlockSize>>>(flockData, tmp);

						cudaEventRecord(stop);
						CHECK(cudaEventSynchronize(stop));
						cudaEventElapsedTime(&milliseconds, start, stop);
						printf("    GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);
					
						//print some boids to check the update correctness 
						for(int i = 0; i < 10; i++){
								printBoid(i, flockData, flockDim);
						}
						for(int i = flockDim/2-5; i < flockDim/2+5; i++){
								printBoid(i, flockData, flockDim);
						}
					  for(int i = flockDim-11; i < flockDim-1; i++){
								printBoid(i, flockData, flockDim);
						}
					
						printf("				Speedup: %.2f\n", cpuTime/(milliseconds / 1000));
					
						printf("\nUpdate correctness: %i\n\n", checkUpdateCorrectness(flockData, flockDataSeq, flockDim));
					
						// update total threads number in constant memory
						cudaMemcpyToSymbol(threadsNumDev, &neighThreadsNum, sizeof(threadsNumDev));
						
						computeAllNeighborhoods<<<neighGridSize, neighBlockSize>>>(flockData, neighborhoods);
					
						tmpTime += updateTime;
						iterations--;

						//std::cout << std::endl;
						//printFlock(flockData, flockDim);	
					
						//std::cout << std::endl;
						//printNeighborhoods(neighborhoodsSeq, flockDim);
				}
		}

		cudaFree(flockData);
		cudaFree(neighborhoods);
	  //CHECK(cudaFree(flockData));
		//CHECK(cudaFree(neighborhoods));
		free(neighborhoodsSeq);
		free(flockDataSeq);
		
		cudaDeviceReset();

		printf("\nEnd.\n");

		return 0;
}

Overwriting FlockingBehaviourPar.cu


In [ ]:
%%writefile Utils.h
#include <vector>
#include <math.h>
#include <chrono>
#include <sys/time.h>
#include <cuda_runtime_api.h>
#include <cuda.h>
#include <stdio.h>

#ifndef UTILS_H
#define UTILS_H

typedef unsigned long ulong;
typedef unsigned int uint;

// returns the distance between the passed vectors
float vector3Distance(const float*, const float*);
float vector3Distance(const float*, const float*, const float*, const float*, const float*, const float*);

// returns the sum of the passed vectors
void vector3Sum(const float*, const float*, float*);
void vector3Sum(const float*, const float*, const float*, const float*, const float*, const float*, float*);

// returns the subtraction of the passed vectors
void vector3Sub(const float*, const float*, float*);
void vector3Sub(const float*, const float*, const float*, const float*, const float*, const float*, float*);

// returns the multiplication of the passed vectors
void vector3Mul(const float*, const float, float*);
void vector3Mul(const float*, const float*, const float*, const float, float*);

// returns the blending of the passed vectors representing directions
void blendDirections(const float*, const float*, const float*, float*);

// returns the magnitude of the passed vector
float vector3Magnitude(const float*);
float vector3Magnitude(const float*, const float*, const float*);

// normalizes the passed vector
void vector3Normalize(float*);
void vector3Normalize(float*, float*, float*);

#define CHECK(call)                                                           \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
    }                                                                          \
}

#define CHECK_CUBLAS(call)                                                     \
{                                                                              \
    cublasStatus_t err;                                                        \
    if ((err = (call)) != CUBLAS_STATUS_SUCCESS)                               \
    {                                                                          \
        fprintf(stderr, "Got CUBLAS error %d at %s:%d\n", err, __FILE__,       \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CURAND(call)                                                     \
{                                                                              \
    curandStatus_t err;                                                        \
    if ((err = (call)) != CURAND_STATUS_SUCCESS)                               \
    {                                                                          \
        fprintf(stderr, "Got CURAND error %d at %s:%d\n", err, __FILE__,       \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CUFFT(call)                                                      \
{                                                                              \
    cufftResult err;                                                           \
    if ( (err = (call)) != CUFFT_SUCCESS)                                      \
    {                                                                          \
        fprintf(stderr, "Got CUFFT error %d at %s:%d\n", err, __FILE__,        \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CUSPARSE(call)                                                   \
{                                                                              \
    cusparseStatus_t err;                                                      \
    if ((err = (call)) != CUSPARSE_STATUS_SUCCESS)                             \
    {                                                                          \
        fprintf(stderr, "Got error %d at %s:%d\n", err, __FILE__, __LINE__);   \
        cudaError_t cuda_err = cudaGetLastError();                             \
        if (cuda_err != cudaSuccess)                                           \
        {                                                                      \
            fprintf(stderr, "  CUDA error \"%s\" also detected\n",             \
                    cudaGetErrorString(cuda_err));                             \
        }                                                                      \
        exit(1);                                                               \
    }                                                                          \
}

inline double seconds() {
    struct timeval tp;
    struct timezone tzp;
    int i = gettimeofday(&tp, &tzp);
    return ((double)tp.tv_sec + (double)tp.tv_usec * 1.e-6);
}

inline void device_name() {
    int dev = 0;
    cudaDeviceProp deviceProp;
    CHECK(cudaGetDeviceProperties(&deviceProp, dev));
    printf("device %d: %s\n", dev, deviceProp.name);
    CHECK(cudaSetDevice(dev));
}
#endif

Overwriting Utils.h


In [ ]:
%%writefile Utils.cpp
#include "Utils.h"

float vector3Distance(const float* v, const float* w){
    return sqrt(pow(w[0] - v[0], 2) + pow(w[1] - v[1], 2) + pow(w[2] - v[2], 2));
}

float vector3Distance(const float* a, const float* b, const float* c, const float* x, const float* y, const float* z){
    return sqrt(pow(a[0] - x[0], 2) + pow(b[0] - y[0], 2) + pow(c[0] - z[0], 2));
}

void vector3Sum(const float* v, const float* w, float* res){
    
    res[0] = v[0] + w[0];
    res[1] = v[1] + w[1];
    res[2] = v[2] + w[2];
}

void vector3Sum(const float* a, const float* b, const float* c, const float* x, const float* y, const float* z, float* res){
    
    res[0] = a[0] + x[0];
    res[1] = b[0] + y[0];
    res[2] = c[0] + z[0];
}

void vector3Sub(const float* v, const float* w, float* res){
    
    res[0] = v[0] - w[0];
    res[1] = v[1] - w[1];
    res[2] = v[2] - w[2];
}

void vector3Sub(const float* a, const float* b, const float* c, const float* x, const float* y, const float* z, float* res){
    
    res[0] = a[0] - x[0];
    res[1] = b[0] - y[0];
    res[2] = c[0] - z[0];
}

void vector3Mul(const float* v, const float n, float* res){
    
    res[0] = v[0] * n;
    res[1] = v[1] * n;
    res[2] = v[2] * n;
}

void vector3Mul(const float* a, const float* b, const float* c, const float n, float* res){
    
    res[0] = a[0] * n;
    res[1] = b[0] * n;
    res[2] = c[0] * n;
}

void blendDirections(const float* v, const float* w, const float* u, float* res){
    
    res[0] = 0;
    res[1] = 0;
    res[2] = 0;
    vector3Sum(res, v, res);
    vector3Sum(res, w, res);
    vector3Sum(res, u, res);

    vector3Normalize(res);
}

float vector3Magnitude(const float* v){
    return sqrt(pow(v[0], 2) + pow(v[1], 2) + pow(v[2], 2));
}

float vector3Magnitude(const float* a, const float* b, const float* c){
    return sqrt(pow(a[0], 2) + pow(b[0], 2) + pow(c[0], 2));
}

void vector3Normalize(float* v){
    if(v[0] != 0 || v[1] != 0 || v[2] != 0){
        float magnitude = vector3Magnitude(v);
        v[0] *= (1/magnitude);
        v[1] *= (1/magnitude);
        v[2] *= (1/magnitude);
    }
}

void vector3Normalize(float* a, float* b, float* c){
    if(a[0] != 0 || b[0] != 0 || c[0] != 0){
        float magnitude = vector3Magnitude(a,b,c);
        a[0] *= (1/magnitude);
        b[0] *= (1/magnitude);
        c[0] *= (1/magnitude);
    }
}

Overwriting Utils.cpp


In [5]:
# Compilation (e.g. -arch=sm_37 if compute capability is 3.7)
!nvcc -arch=sm_37 FlockingBehaviourPar.cu FlockingBehaviourSeq.cpp Utils.cpp -lcurand -o FlockingBehaviourPar

# Execution
!./FlockingBehaviourPar

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
device 0: Tesla K80

Flock dimension: 40960
Neighborhood dimension: 50.00
Velocity: 20.00
Update time: 1.50
Iterations: 1
Separation weight: 1.00
Cohesion weight: 1.00
Align weight: 1.00
Minimum random number: -50000
Maximum random number: 50000
Decimal digits of random numbers: 3.00

Needed threads number: 2048
Threads used: 2048
Block size: 128
Grid size: 16
Generations per thread: 20
Generations of last thread: 20


GPU Flock generation...
    GPU elapsed time: 0.00838 (sec)
Boid 0: pos(-38.61, -37.65, -46.24); dir(-0.6677, -0.3015, 0.6807)
Boid direction magnitude: 1.00000
Boid 1: pos(-12.25, -6.901, 25.25); dir(-0.9637, -0.2232, -0.1465)
Boid direction magnitude: 1.00000
Boid 2: pos(-39.82, -43.09, -12.38); dir(-0.1309, -0.6239, 0.7704)
Boid direction magnitude: 1.00000
